<a href="https://colab.research.google.com/github/LordRelentless/NGFTSimulations/blob/main/Simulation_1_2_Mass_Energy_Information_Equivalence_in_Particle_Decayv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
module NGFTDecay

using Random, Printf

# Constants
const k = 1.38e-23        # Boltzmann constant
const a_U = 1e-21         # NGFT information-energy conversion
const c = 3e8             # Speed of light

# Particle Struct
mutable struct Particle
    id::Int
    mass::Float64
    velocity::Tuple{Float64, Float64}
    parent_id::Union{Int, Nothing}
    generation::Int
end

# Ledger to track all particles and events
mutable struct SimulationState
    particles::Dict{Int, Particle}
    next_id::Int
end

function SimulationState()
    return SimulationState(Dict{Int, Particle}(), 1)
end

# Calculate total energy from information
function total_information(p::Particle)
    vx, vy = p.velocity
    ke = 0.5 * p.mass * (vx^2 + vy^2)
    rest = p.mass * c^2
    info_rest = rest / ((k + 1) * a_U)
    info_kinetic = ke / ((k + 1) * a_U)
    return info_rest + info_kinetic
end

# Add particle to state
function add_particle!(state::SimulationState, p::Particle)
    state.particles[p.id] = p
    state.next_id += 1
end

# Decay rule: splits particle into two equal daughters with opposite momentum
function decay!(state::SimulationState, pid::Int)
    parent = state.particles[pid]

    m_d = parent.mass / 2
    speed = rand() * 2.0  # Randomized kinetic energy
    θ = rand() * 2π
    vx = speed * cos(θ)
    vy = speed * sin(θ)

    d1 = Particle(state.next_id, m_d, (vx, vy), parent.id, parent.generation + 1)
    add_particle!(state, d1)

    d2 = Particle(state.next_id, m_d, (-vx, -vy), parent.id, parent.generation + 1)
    add_particle!(state, d2)
end

# Run chain simulation for N generations
function run_decay_chain(generations::Int)
    state = SimulationState()
    parent = Particle(state.next_id, 4.0, (0.0, 0.0), nothing, 0)
    add_particle!(state, parent)

    for g in 1:generations
        current = [p for p in values(state.particles) if p.generation == g - 1]
        for p in current
            decay!(state, p.id)
        end
    end

    return state
end

# Audit information totals
function audit_information(state::SimulationState)
    gen_info = Dict{Int, Float64}()

    for p in values(state.particles)
        I = total_information(p)
        gen = p.generation
        gen_info[gen] = get(gen_info, gen, 0.0) + I
    end

    println("=== Informational Energy by Generation ===")
    for g in sort(collect(keys(gen_info)))
        @printf("Gen %d: %.4e bits\n", g, gen_info[g])
    end
end

# Run it
state = run_decay_chain(3)
audit_information(state)

end # module

=== Informational Energy by Generation ===
Gen 0: 3.6000e+38 bits
Gen 1: 3.6000e+38 bits
Gen 2: 3.6000e+38 bits
Gen 3: 3.6000e+38 bits


Main.NGFTDecay